In [1]:
!pip install transformers==2.8.0
!pip install --upgrade PyYAML==5.1.2  # old version couldn't sort the order of dictionary

     |████████████████████████████████| 563 kB 9.0 MB/s 
     |████████████████████████████████| 5.6 MB 37.2 MB/s 
     |████████████████████████████████| 1.2 MB 53.2 MB/s 
     |████████████████████████████████| 895 kB 26.8 MB/s 
     |████████████████████████████████| 131 kB 68.3 MB/s 
     |████████████████████████████████| 7.8 MB 27.2 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 138 kB 73.0 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 265 kB 7.9 MB/s

# New Section

In [2]:
!pip show transformers

Name: transformers
Version: 2.8.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: boto3, sentencepiece, filelock, tokenizers, tqdm, regex, sacremoses, requests, numpy
Required-by: 


In [3]:
import torch
print(torch.__version__)

1.9.0+cu102


In [4]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)


model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

sentence = "it is not the item I expected"
# sentence = "What are the ingredients required to bake a perfect cake?"
# sentence = "What is the best possible approach to learn aeronautical engineering?"
# sentence = "Do apples taste better than oranges in general?"


text =  "paraphrase: " + sentence + " </s>"


max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=20
)


print ("\nOriginal Question ::")
print (sentence)
print ("\n")
print ("Paraphrased Questions :: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

device  cuda


Sequence can't be padded as no maximum length is specified and the model maximum length is too high.



Original Question ::
it is not the item I expected


Paraphrased Questions :: 
0: I lost some items on my list and it is not what I expected. It is the item I expected.
1: I noticed this item is not the items I expected, but not the items I expected. If the item I was expecting was in poor condition, there are other things to be disappointed in. I will replace it in a month or two for a full refund.
2: I'm sorry to say this is not the item I expected. This is something I didn't expect.
3: It was not my expectation or as expected. This is not a good item. I just needed it.
4: I've seen one and it's not the item I expected. I have not read any new and it is not my original product.
5: Is something I did expect to be able to sell?
6: Is this item not what I expected to be?
7: I don't think it is the item I've expected, but it is to be expected. It is not the item I expected.
8: Is this what I expected but is not as good and efficient as expected?
9: Is the item being shipped wrong (unexp

In [5]:
'k-fold'

import numpy as np
from sklearn.model_selection import KFold
X = np.array(['a','b','c','d','e'])
y = np.array([1,2,3,4,5])
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)
KFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in kf.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  


KFold(n_splits=5, random_state=None, shuffle=False)
TRAIN: [1 2 3 4] TEST: [0]
TRAIN: [0 2 3 4] TEST: [1]
TRAIN: [0 1 3 4] TEST: [2]
TRAIN: [0 1 2 4] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]


# clean data list



In [ ]:
# read gpt3 data
import yaml
with open('/content/gpt3_nlu.yml','r') as f:
  data=yaml.safe_load(f)

# text=[]
# for i in data['nlu']:
#   for j in i['examples'].split('\n'):     # get the clean data from yaml
#     input=j.replace('- ','')
#     text.append(input)


In [ ]:
c=data  # gpt3 data is stored in a list of dictionary

In [ ]:
for i in data:
  print(i)


faq_account_sign_in
faq_animal_welfare
faq_change_password
faq_delivery_address
faq_delivery_slot
faq_enquire_premier
faq_exchange_item
faq_faulty_item
faq_fur
faq_gift_voucher
faq_marketplace_app
faq_missing_comms
faq_missing_item
faq_next_day_delivery
faq_order_not_received
faq_payment_card_refund
faq_payment_currency
faq_payment_options
faq_refund_delivery_charge
faq_responsible_shopping
faq_return_format
faq_return_multiple_orders
faq_return_prohibited_items
faq_return_received
faq_return_slip
faq_returns_policy
faq_size_guide
faq_stock
faq_student_discount
faq_track_delivery
faq_unsubscribe_email
faq_wrong_item_received


In [ ]:

# for the format for Rasa framework
import yaml
with open('/content/nlu_fake.yml','r') as f:
  data=yaml.safe_load(f)



split yaml file 80% 20%

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
X = np.array(['a','b','c','d','e'])
y = np.array([1,2,3,4,5])
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)
KFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in kf.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]

KFold(n_splits=5, random_state=None, shuffle=False)
TRAIN: [1 2 3 4] TEST: [0]
TRAIN: [0 2 3 4] TEST: [1]
TRAIN: [0 1 3 4] TEST: [2]
TRAIN: [0 1 2 4] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X

array('2.0', dtype='<U3')

In [ ]:
f_train=[[],[],[],[],[]]
f_test=[[],[],[],[],[]]
# print(f_train)
kf = KFold(n_splits=5)

for i in c:
 
  X=np.array(c[i])
  kf.get_n_splits(X)

  
  KFold(n_splits=5, random_state=1, shuffle=True)
  num=0
  for train_index, test_index in kf.split(X):
    s1=[]
    s2=[]
    
    s1.append(X[train_index].tolist())
    # print('--------')
    # print(s1)
    s2.append(X[test_index].tolist())
    f_train[num].append(s1)
    f_test[num].append(s2)
    num=num+1
  
  

  # train,test=train_test_split(section, test_size=0.2, random_state=42)
  # f_train.append(train)  #80%
  # f_test.append(test)  #20%


In [ ]:
!mkdir gpt_nlu_original

In [ ]:
!zip -r /content/gpt_nlu_original.zip /content/gpt_nlu_original

  adding: content/gpt_nlu_original/ (stored 0%)
  adding: content/gpt_nlu_original/1cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/1cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/3cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/4cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/3cv_original_test.yml (deflated 76%)
  adding: content/gpt_nlu_original/4cv_original_test.yml (deflated 75%)


In [ ]:
for element in f_train:
  for i in element:
    print(i[0])
   

In [ ]:
count=0
for element in f_train:
  count=count+1
  for (i,j) in zip(element,data['nlu']):

    
    
    
    j['examples']=i[0]

  
  with open('/content/gpt_nlu_original/{}cv_original_train.yml'.format(count), 'w') as outfile:
      yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)

In [ ]:
count=0
for element in f_test:
  count=count+1
  for (i,j) in zip(element,data['nlu']):

    
    
    
    j['examples']=i[0]

  
  with open('/content/gpt_nlu_original/{}cv_original_test.yml'.format(count), 'w') as outfile:
      yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)

In [ ]:
count=0
for element in f_test:
  count=count+1
  for (i,j) in zip(element,c['nlu']):
    
    l=[]
    for k in i[0]:
      print(k)
      l.append(k.replace('- ',''))
      
    j['examples']=l

  print(c['nlu'])
  with open('/content/{}cv_original_test.yml'.format(count), 'w') as outfile:
      yaml.dump(c, outfile, default_flow_style=False, sort_keys=False)

# work on the augumented data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import yaml
with open('/content/drive/MyDrive/hf_asos/ayo-faq/data/1cv_original_train.yml','r') as f:
  data=yaml.safe_load(f)


f_train=[]
for i in data['nlu']:
  text=[]
  for j in i['examples'].split('\n'):     # get the clean data from yaml
    input=j.replace('- ','')
    text.append(input)
  f_train.append(text[:-1])

In [ ]:
f_train

In [ ]:
def generate_sentences(sentence,num,k,p):  #num: number of augumented sentence


  text =  "paraphrase: " + sentence + " </s>"
  print(text)


  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

             
  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=k,
      top_p=p,
      early_stopping=True,
      num_return_sequences=30
  )
  final_outputs=[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)

  return final_outputs[0:num]

In [ ]:
generate_sentences('when can i get my delivery?',10,150,0.87)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['When is delivery due?',
 'When can I get delivery?',
 'When can I get delivery online?',
 'How soon can I expect my delivery?',
 'When will I get my delivery?',
 'When will you send your deliveries?',
 'When will you get your delivery?',
 'When can I expect my delivery?',
 'How do I get delivery?',
 'How long will it take to get a delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,50,0.99)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['How do I get my delivery?',
 'How soon can I order delivered?',
 'When can I get my delivery on schedule?',
 'When can I have delivery?',
 'When will be my delivery?',
 'When will a delivery be done?',
 'How can I get my delivery?',
 'How long can I have my delivery?',
 'When do you get the delivery?',
 'When can I expect my delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,50,0.89)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['When will I receive my delivery?',
 'How should I know when I get my delivery?',
 'When can you get delivery?',
 'How do I get delivery?',
 'When can I have delivery?',
 'How do I get my delivery?',
 'How long will it take for delivery?',
 'When will you get your delivery?',
 'When will I get my delivery?',
 'How can you get delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,100,0.89)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['When do I get delivery?',
 'When can I get delivery service?',
 'How can I get delivery?',
 'How can I get my delivery to the house?',
 'How soon can I expect delivery?',
 'When will you get your delivery?',
 'When will I get my delivery?',
 'How often do I get my delivery?',
 'How can I get delivery information?',
 'When can I expect my delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,100,0.99)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['How do I get my delivery?',
 'How can I get delivery?',
 'When can I start my delivery?',
 'Where can I get delivery?',
 'How many days will you wait for delivery if you are not using your delivery service?',
 'Where do I get delivery?',
 'How quickly do I get my delivery order?',
 'When can I get delivery?',
 'How can I get my delivery?',
 'When will I receive my delivery?']

In [ ]:
generate_sentences('when can i get my delivery?',10,150,0.99)

Sequence can't be padded as no maximum length is specified and the model maximum length is too high.


paraphrase: when can i get my delivery? </s>


['When do I have to have delivery?',
 'When will I get my delivery?',
 'How can I get delivery?',
 'How soon should I get delivery?',
 'How do I get delivery day?',
 'How can I have my delivery?',
 'When do we get delivery?',
 'How can I get delivery today?',
 'How can I get my delivery anywhere near me?',
 'When is delivery coming?']

In [ ]:
  with open('/content/improved.yml', 'w') as outfile:
      yaml.dump(c, outfile, default_flow_style=False, sort_keys=False)

In [ ]:
for i in data['nlu']:
  print(i['examples'].split('\n'))

In [ ]:
store[-1][-1][9]

In [ ]:
!zip -r /content/cv.zip /content/cv1

In [ ]:
print('123')

In [ ]:
c['nlu']

In [ ]:
import yaml
with open('/content/nlu.yml') as f:
  data=yaml.safe_load(f)


f_train=[]
for i in data['nlu']:
  text=[]
  for j in i['examples'].split('\n'):     # get the clean data from yaml
    input=j.replace('- ','')
    text.append(input)
  f_train.append(text[:-1])

In [ ]:
for i in data['nlu']:
  print(i['intent'])

faq_fur
faq_delivery_slot
faq_delivery_address
faq_return_prohibited_items
faq_exchange_item
faq_return_multiple_orders
faq_track_delivery
faq_enquire_premier
faq_responsible_shopping
faq_refund_delivery_charge
faq_return_received
faq_return_slip
faq_payment_options
faq_unsubscribe_email
faq_payment_currency
faq_return_format
faq_gift_voucher
faq_next_day_delivery
faq_account_sign_in
faq_missing_item
faq_change_password
faq_faulty_item
faq_wrong_item_received
faq_student_discount
faq_missing_comms
faq_payment_currency
faq_payment_card_refund
faq_animal_welfare
faq_returns_policy
faq_order_not_received
faq_size_guide
faq_marketplace_app
faq_stock


In [ ]:
! mkdir cv1

In [ ]:
import yaml
with open('/content/drive/MyDrive/hf_asos/ayo-faq/data/gpt_nlu_original/1cv_original_train.yml'.format(qq),'r') as f:
  data=yaml.safe_load(f)


f_train=[]
for i in data['nlu']:
  text=[]
  for j in i['examples'].split('\n'):     # get the clean data from yaml
    input=j.replace('- ','')
    text.append(input)
  f_train.append(text[:-1])

NameError: ignored

In [ ]:
for qq in range(1,6):

  import yaml
  with open('/content/drive/MyDrive/hf_asos/ayo-faq/data/gpt_nlu_original/{}cv_original_train.yml'.format(qq),'r') as f:
    data=yaml.safe_load(f)


  f_train=[]
  for i in data['nlu']:
    text=[]
    for j in i['examples'].split('\n'):     # get the clean data from yaml
      input=j.replace('- ','')
      text.append(input)
    f_train.append(text[:-1])

################################# read file below    
  c=data
  store=np.ones(32).tolist()
  import numpy as np
    

  count=0
  for i in f_train:
  
    arr=[]
    for sentence in i:
        l=[]
        sentences=generate_sentences(sentence,12,150,0.87)
        print('----------------')
        print(sentence)
        l.append(sentence)
        l.extend(sentences) 
        new=[]
        print(l)
        ## store sentence
        for q in range(1,11):
            
          new.append(l[0:q+1])
        arr.append(new)
        
        
    store[count]=arr
    count=count+1


      ############################################################################
  d=[]
  for i in store:
    print(np.array(i).shape )
    d.append(np.array(i).shape[0])   



  for num in range(0,10):
    c=data
      
    count=0
    for (i,j,n) in zip(f_train,c['nlu'],d):
      new=[]
      print('*************')
      print(j['intent'])
      for q in range(n):
        new.extend(store[count][q][num])
          
            
          
      
        
        
      count=count+1
      print('-----------')
      print(len(new))
      j['examples']=new
        
    print(c['nlu'])
      

    with open('/content/drive/MyDrive/gpt3/{}cv_{}augumented_fixed.yml'.format(qq,num+1), 'w') as outfile:
          yaml.safe_dump(c, outfile, default_flow_style=False, sort_keys=False)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import numpy as np

In [ ]:
from google.colab import files
files.download("/content/cv.zip")

In [ ]:
for i in store[0]:
  print(i)

In [ ]:
d

In [ ]:
d=[]
for i in store:
  print(np.array(i).shape )
  d.append(np.array(i).shape[0])   


qq=1
for num in range(0,10):
  c=data
    
  count=0
  for (i,j,n) in zip(f_train,c['nlu'],d):
    new=[]
    print('*************')
    print(j['intent'])
    for q in range(n):
      new.extend(store[count][q][num])
        
          
        
    
      
      
    count=count+1
    print('-----------')
    print(len(new))
    j['examples']=new
      
  print(c['nlu'])
    

  with open('/content/cv1/{}cv_{}augumented_fixed.yml'.format(qq,num+1), 'w') as outfile:
        yaml.safe_dump(c, outfile, default_flow_style=False, sort_keys=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


(10, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
(8, 10)
*************
faq_fur
-----------
20
*************
faq_delivery_slot
-----------
16
*************
faq_delivery_address
-----------
16
*************
faq_return_prohibited_items
-----------
16
*************
faq_exchange_item
-----------
16
*************
faq_return_multiple_orders
-----------
16
*************
faq_track_delivery
-----------
16
*************
faq_enquire_premier
-----------
16
*************
faq_responsible_shopping
-----------
16
*************
faq_refund_delivery_charge
-----------
16
*************
faq_return_received
-----------
16
*************
faq_return_slip
-----------
16
*************
faq_payment_options
-----------
16
*************
faq_unsubscribe_email
-----------
16
*************
faq_payment_currency
-----

In [ ]:
c=data

qq=0
store=np.ones(33).tolist()  #length33  store a nested list
for element in f_train:  # 33
  qq=qq+1 
  import numpy as np
  

  count=0
  for i in f_train:
    l=[]
    for sentence in i:
        arr=[]
        sentences=generate_sentences(sentence,10,150,0.87)
        print('----------------')
        print(sentence)
        l.append(sentence)
        l.extend(sentences) 
        new=[]
        print(l)
        ## store sentence
        for q in range(1,11):
          
          new.append(l[0:q+1])
        arr.append(new)
      
      
    store[count]=arr
    count=count+1
          # with open('/content/{}a.yml'.format(q), 'a') as outfile:
          #     yaml.dump(l, outfile, default_flow_style=False, sort_keys=False)

  #   ############################################################################
  # d=[]
  # for i in store:
  #   print(np.array(i).shape )
  #   d.append(np.array(i).shape[0])    

  #   ##############################################################################
  #   # write data from the stored list and wirte in 10 files
  # for num in range(0,10):
  #   c=data
      
  #   count=0
  #   for (j,n) in zip(c['nlu'],d):
  #     new=[]
        
  #     for q in range(n):
  #       new.extend(store[count][q][num])
          
            
          
      
        
        
  #     count=count+1
        
  #     j['examples']=new
        
  #   print(c['nlu'])
      

  #   with open('/content/cv/{}cv_{}augumented_fixed.yml'.format(qq,num+1), 'w') as outfile:
  #       yaml.safe_dump(c, outfile, default_flow_style=False, sort_keys=False)

In [ ]:
np.array(store).shape

In [ ]:
np.array(store)[0]

In [ ]:
f_train=data['nlu']






In [ ]:
for qq in range(1,6):

  import yaml
  with open('/content/drive/MyDrive/hf_asos/ayo-faq/data/gpt_nlu_original/{}cv_original_train.yml'.format(qq),'r') as f:
    data=yaml.safe_load(f)


  f_train=[]
  for i in data['nlu']:
    text=[]
    for j in i['examples'].split('\n'):     # get the clean data from yaml
      input=j.replace('- ','')
      text.append(input)
    f_train.append(text[:-1])

################################# read file below    
  c=data
  store=np.ones(32).tolist()
  import numpy as np
    

  count=0
  for i in f_train:
  
    arr=[]
    for sentence in i:
        l=[]
        sentences=generate_sentences(sentence,12,150,0.87)
        print('----------------')
        print(sentence)
        l.append(sentence)
        l.extend(sentences) 
        new=[]
        print(l)
        ## store sentence
        for q in range(1,11):
            
          new.append(l[0:q+1])
        arr.append(new)
        
        
    store[count]=arr
    count=count+1


      ############################################################################
  d=[]
  for i in store:
    print(np.array(i).shape )
    d.append(np.array(i).shape[0])   



  for num in range(0,10):
    c=data
      
    count=0
    for (i,j,n) in zip(f_train,c['nlu'],d):
      new=[]
      print('*************')
      print(j['intent'])
      for q in range(n):
        new.extend(store[count][q][num])
          
            
          
      
        
        
      count=count+1
      print('-----------')
      print(len(new))
      j['examples']=new
        
    print(c['nlu'])
      

    with open('/content/drive/MyDrive/gpt3/{}cv_{}augumented_fixed.yml'.format(qq,num+1), 'w') as outfile:
          yaml.safe_dump(c, outfile, default_flow_style=False, sort_keys=False)

# Change the gpt3 data into format for RASA

In [ ]:
import yaml
with open('/content/gpt3_nlu.yml','r') as f:
  data=yaml.safe_load(f)


c=data

FileNotFoundError: ignored

In [ ]:
type(c)

NameError: ignored

In [ ]:

# for the format for Rasa framework
import yaml
with open('/content/nlu.yml','r') as f:
  data=yaml.safe_load(f)



In [ ]:
!mkdir gpt_nlu_original

In [ ]:

for i in data['nlu']:


  key=i['intent']
   
    
    
  i['examples']=c[key]

  
with open('/content/gpt_nlu_original/gpt_all_generated.yml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)

# Add gpt3 augmented data into 5-fold original files

In [6]:
import numpy as np

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
! mkdir gpt3

In [10]:
# read and clean gpt data
import yaml
with open('/content/gpt_rasa.yml','r') as f:
  data_gpt=yaml.safe_load(f)


gpt=[]
for i in data_gpt['nlu']:
  text=[]
  for j in i['examples'].split('\n'):     # get the clean data from yaml
    input=j.replace('- ','')
    text.append(input)
  gpt.append(text[:-1])

In [11]:
for qq in range(1,6):

  import yaml
  with open('/content/drive/MyDrive/hf_asos/ayo-faq/data/original_data_gpt/{}cv_original_train.yml'.format(qq),'r') as f:
    data=yaml.safe_load(f)


  f_train=[]
  for i in data['nlu']:
    text=[]
    for j in i['examples'].split('\n'):     # get the clean data from yaml
      input=j.replace('- ','')
      text.append(input)
    f_train.append(text[:-1])

################################# read file below    
  c=data
  store=np.ones(32).tolist()
  import numpy as np
    

  count=0
  for (i,j) in zip(f_train,gpt):
  
    arr=[]
    # for sentence in i:
    #     l=[]
    #     sentences=
    #     print('----------------')
    #     print(sentence)
    #     l.append(sentence)
    #     l.extend(sentences) 
    #     new=[]
    #     print(l)
    #     ## store sentence
    pool=np.array(j)
    np.random.shuffle(pool)
    j=pool.tolist()
    new=[]
    for q in range(1,11):

 
      new.append(j[0:10*q])
    arr.append(new)
        
        
    store[count]=arr
    count=count+1


      ############################################################################
  d=[]
  for i in store:
    print(np.array(i).shape )
    d.append(np.array(i).shape[0])   



  for num in range(0,10):
    c=data
      
    count=0
    for (i,j,n) in zip(f_train,c['nlu'],d):
      new=[]
      print('*************')
      print(j['intent'])

      for q in range(n):

        new.extend(store[count][q][num])
          
            
          
      
        
        
      count=count+1
      shuffle=np.array(new)
      np.random.shuffle(shuffle)
      new=shuffle.tolist()
      j['examples']=new
        
    print(c['nlu'])
      

    with open('/content/drive/MyDrive/gpt3/{}cv_{}augumented_fixed.yml'.format(qq,num+1), 'w') as outfile:
          yaml.safe_dump(c, outfile, default_flow_style=False, sort_keys=False)
     

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
np.array(store).shape

(32, 1, 10, 10)

In [ ]:
store[0][0]

In [ ]:
!zip -r /content/gpt_nlu_original.zip /content/gpt_nlu_original

  adding: content/gpt_nlu_original/ (stored 0%)
  adding: content/gpt_nlu_original/1cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/1cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/3cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/4cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/5cv_original_test.yml (deflated 75%)
  adding: content/gpt_nlu_original/2cv_original_train.yml (deflated 80%)
  adding: content/gpt_nlu_original/3cv_original_test.yml (deflated 76%)
  adding: content/gpt_nlu_original/4cv_original_test.yml (deflated 75%)


**fine tune the model**,  
based on the function csim, evaluate the average the score for different top_p and top_k in the t5 generator

In [ ]:
sentence='when can i get my parcel'
topk=np.arange(50,200,20)
topp=np.arange(0.8,1,0.01)
for k in topk:
  for p in topp:
    result=generate_sentences(sentence,10,int(k),p)
    score=[]
    for i in result:
      score.append(csim(sentence,i))

    s=np.mean(np.array(score))
    if s>60.0:
      print('top_k: {}, top_p: {}, average score: {}'.format(k,p,s))

In [ ]:



sentence='i am annoyed with you'
generate_sentences(sentence,10,150,0.87)

In [ ]:
import numpy as np
score=[]
for i in result:
  score.append(csim(sentence,i))

np.mean(np.array(score))

evaluation score

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "bert-base-cased-finetuned-mrpc"
tokenizer_s = AutoTokenizer.from_pretrained(model_name)
model_score = AutoModelForSequenceClassification.from_pretrained(model_name)
def csim(sequence_0,sequence_1):
  
  tokens = tokenizer_s.encode_plus(sequence_0, sequence_1, return_tensors="pt")
  classification_logits = model_score(**tokens)[0]
  results = torch.softmax(classification_logits, dim=1).tolist()[0]
  return round(results[1] * 100)
  
      
      

In [ ]:
print(f_train)

generate data and wirite into files

1 original examples + T5 data

In [ ]:
for num in range(1,11):
  for (i,j) in zip(f_train,c['nlu']):
    l=[]
    for k in i:
      if k == "":
        continue
      else:
        sentence=k.replace('- ','')

        
        sentences=generate_sentences(sentence,num,150,0.86)
        l.append(sentence)
        l.extend(sentences)
        
        
    j['examples']=l

  print(c['nlu'])

  with open('/content/{}augumented_training_improved.yml'.format(num), 'w') as outfile:
      yaml.dump(c, outfile, default_flow_style=False, sort_keys=False)  

In [ ]:
for num in range(10,11):
  for (i,j) in zip(f_train,c['nlu']):
    l=[]
    for k in i:
      if k == "":
        continue
      else:
        sentence=k.replace('- ','')

        
        sentences=generate_sentences(sentence,num,150,0.86)
        # l.append(sentence)
        l.extend(sentences)
        
        
    j['examples']=l

  print(c['nlu'])

  with open('improved.yml'.format(num), 'w') as outfile:
      yaml.dump(c, outfile, default_flow_style=False, sort_keys=False)  

In [ ]:
import numpy as np
l


In [ ]:
import yaml

print(yaml.__version__)

In [ ]:
data['nlu'][0]['examples']='/n'.join(data['nlu'][0]['examples'])

In [ ]:
data['nlu'][0]

In [ ]:
with open('/content/original_Test.yml', 'w') as outfile:
    yaml.dump(data['nlu'][0]['examples'], outfile, default_flow_style=False, sort_keys=False, default_style='|')

In [ ]:
for i in data['nlu']:
  print(i)

In [ ]:
for i in c['nlu']:
  print

In [ ]:
    a='|good afternoon/nWhat does it feel like to have a lovely afternoon today?/ngood evening/n'

In [ ]:
data['nlu'][0]['examples']=a

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)


model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/training_data/

In [ ]:
import pandas as pd
raw_data = pd.read_excel(r'RASA_Data.xlsx')
print(raw_data.head())

In [ ]:
pure_data = raw_data[['Text','Intend']]
pure_data = pure_data.iloc[0:477,:]
print(pure_data)

In [ ]:
final_table = pd.DataFrame(columns=['Text','Corresponding','Intend'])

print(pure_data.shape[0])

for i in range(pure_data.shape[0]):
  print(iter)
  temp_text = pure_data.loc[i,'Text']
  temp_intend = pure_data.loc[i, 'Intend']
  temp_example = gene_sent(temp_text,1,temp_intend)
  final_table = pd.concat([final_table, temp_example])

print(final_table)

In [ ]:
final_table = final_table.reset_index(drop=True)

In [ ]:
final_table.to_excel(r'gene_data.xlsx')